<a href="https://colab.research.google.com/github/Bainer13/Data_BBVA_Team_1/blob/Rodri/Proyecto_bootcamp_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Armado del entorno de trabajo**



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [ ]:
ls

 address_bootcamp.snappy.parquet           nationality.snappy.parquet
 customer_basics_bootcamp.snappy.parquet   phones_bootcamp.snappy.parquet
 emails_bootcamp.snappy.parquet            sample_data/
'Marital status.snappy.parquet'            spark-3.3.2-bin-hadoop3.tgz


In [ ]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [ ]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [ ]:
!readlink -f $(which java) | sed "s:bin/java::"


/usr/lib/jvm/java-11-openjdk-amd64/


In [ ]:
!echo $JAVA_HOME

In [ ]:
!pip install -q findspark

In [ ]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 28.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=7f798efe1ae0c4a04aa7c3b7724bc3a2f4fd7b9e6f454fbc87bd8c0eb8ce58a7
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
import pyspark.sql.functions as F

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import col
from pyspark.sql.functions import *
from pyspark.sql.functions import lit
from pyspark.sql.functions import coalesce, expr

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'CORRECTED')

## **Extraccion de datos**

In [ ]:
#Lectura de parquets
df_address = spark.read.parquet('/content/address_bootcamp.snappy.parquet')
df_customer_basics = spark.read.parquet('/content/customer_basics_bootcamp.snappy.parquet')
df_email = spark.read.parquet('/content/emails_bootcamp.snappy.parquet')
df_phones = spark.read.parquet('/content/phones_bootcamp.snappy.parquet')
df_nationality = spark.read.parquet('/content/nationality.snappy.parquet')
df_marital_status_type = spark.read.parquet('/content/Marital status.snappy.parquet')

In [ ]:
df_phones.show()

+-----------+--------------+-------------------+-----------------+----------+----------------+---------------+-------------+-------------------+-----------------+-------------+--------------+---------------+-----------------------+------------------+---------------------+--------------------+----------+-----------+--------------------------+---------------------+--------------------+---------------------+----------------+-------------------------+-------------------------+------------------+-------------------+-----------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+---------------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|phone_country_id|prefix_phone_id|phone_area_id|cellph

## **JOINS ENTRE TABLAS (BBVAW2T1-24/25/26)**

In [ ]:
#Cambio formato de columna
df_phones = df_phones.withColumn("phone_area_id", F.col("phone_area_id").cast("integer"))
df_phones = df_phones.withColumn("phone_number", F.concat(F.lit("+"), F.col("prefix_phone_id"), F.lit(" "), F.col("phone_area_id"), F.lit(" "), F.col("cellphone_prefix_id"), F.lit(" "), F.col("phone_exchange_id"), F.col("phone_line_id")))

In [ ]:
#Join entre clientes y telefonos
df_customer_basics_phones = df_customer_basics.join(df_phones, ['customer_id'], how='left').select(df_customer_basics.customer_id, df_phones.phone_type, df_phones.phone_number, df_phones.last_change_date)
df_customer_basics_phones.sort(F.asc("customer_id")).show(10)

+-----------+----------+-------------------+----------------+
|customer_id|phone_type|       phone_number|last_change_date|
+-----------+----------+-------------------+----------------+
|   00000442|         3| +54 297 15 5296284|      2022-07-20|
|   00000442|         3| +54 297 15 4729337|      2022-07-20|
|   00001419|         3| +54 11 15 66793207|      2022-11-01|
|   00001939|         3| +54 11 15 38700150|      2022-09-21|
|   00002575|      null|               null|            null|
|   00002707|         1|   +54 299  4477116|      2022-08-17|
|   00002790|         3| +54 342 15 4662478|      2022-11-18|
|   00002790|         1|   +54 342  4883620|      2022-10-08|
|   00004287|         3|+54 3541 15 5988799|      2022-08-29|
|   00004724|         3| +54 11 15 24084447|      2022-11-04|
+-----------+----------+-------------------+----------------+
only showing top 10 rows



In [ ]:
#Join entre clientes e email
df_customer_basics_email = df_customer_basics.join(df_email,['customer_id'], how='left').select(df_customer_basics.customer_id, df_email.email_desc, df_email.last_change_date)
df_customer_basics_email.show()

+-----------+--------------------+----------------+
|customer_id|          email_desc|last_change_date|
+-----------+--------------------+----------------+
|   07867437|                null|            null|
|   06813266|                null|            null|
|   29604216|                null|            null|
|   28028529|                null|            null|
|   28366580|                null|            null|
|   02746969|                null|            null|
|   23441482|                null|            null|
|   23610540|                null|            null|
|   25935956|                null|            null|
|   25848977|                null|            null|
|   01957163|                null|            null|
|   06338965|                null|            null|
|   29676364|                null|            null|
|   26205399|   DIONISI@YAHOO.COM|      2019-12-27|
|   03933825|DIGITALIZACION@EE...|      1900-01-01|
|   29416245|                null|            null|
|   29497813

In [ ]:
df_address = df_address.withColumn("full_address", F.concat(F.col("street_name"), F.lit(" "), F.concat(F.col("address_outdoor_id"), F.lit(" "), F.col("address_indoor_id"), F.lit(" "), F.col("indoor_number"), F.col("address_town_name"), F.lit(" "), F.col("address_department_name"), F.lit(" "), F.col("zipcode_id"))))

In [ ]:
#join entre clientes y direccion
df_customer_basics_address = df_customer_basics.join(df_address,['customer_id'], how='left').select(df_customer_basics.customer_id, 'full_address', df_address.last_change_date)
df_customer_basics_address.show()

+-----------+--------------------+----------------+
|customer_id|        full_address|last_change_date|
+-----------+--------------------+----------------+
|   07867437|ALTE FRANCISCO SE...|      2009-01-26|
|   06813266|EL TORDO         ...|      2009-01-26|
|   29604216|                null|            null|
|   28028529|AMENEDO          ...|      2009-01-26|
|   28366580|INCA AAA         ...|            null|
|   28366580|INCA             ...|      2011-07-01|
|   02746969|MERCEDES         ...|      2009-01-26|
|   23441482|AV 39            ...|      2011-07-01|
|   23610540|                null|            null|
|   25935956|AV FERNANDEZ DE L...|      2009-01-26|
|   25848977|                null|            null|
|   01957163|JOSE M DE MINOGUY...|      2009-01-26|
|   06338965|MADERO DR FRANCIS...|      2009-01-26|
|   29676364|                null|            null|
|   26205399|SUIPACHA         ...|      2009-01-26|
|   03933825|HENRY DUNANT     ...|      2009-01-26|
|   29416245

## **FILTRADO Y REDUCCION DE DATOS --> PHONES (BBVAW2T1-27/61)**

In [ ]:
#Hago una ventana por el customer_id y ordenado por la ultima actualizacion 
w = Window.partitionBy(df_customer_basics_phones['customer_id']).orderBy("last_change_date")

In [ ]:
#Agrego columna que cuente la cantidad de veces que se repite el ID
df_customer_basics_phones = df_customer_basics_phones.withColumn("row_num", row_number().over(w))

In [ ]:
#Corto el dataframe quedandome con las ultimas 3 actualizaciones
df_customer_basics_phones = df_customer_basics_phones.filter(col("row_num") <= 3)

In [ ]:
#Reacomodo la tabla con las columnas de interes y concatenando una nueva
df_customer_basics_phones = df_customer_basics_phones.select('customer_id', 'phone_type', 'phone_number', 'row_num')
df_customer_basics_phones.show(10)

+-----------+----------+-------------------+-------+
|customer_id|phone_type|       phone_number|row_num|
+-----------+----------+-------------------+-------+
|   00000442|         3| +54 297 15 5296284|      1|
|   00000442|         3| +54 297 15 4729337|      2|
|   00001419|         3| +54 11 15 66793207|      1|
|   00001939|         3| +54 11 15 38700150|      1|
|   00002575|      null|               null|      1|
|   00002707|         1|   +54 299  4477116|      1|
|   00002790|         1|   +54 342  4883620|      1|
|   00002790|         3| +54 342 15 4662478|      2|
|   00004287|         3|+54 3541 15 5988799|      1|
|   00004724|         3| +54 11 15 24084447|      1|
+-----------+----------+-------------------+-------+
only showing top 10 rows



In [ ]:
#creo una funcion pivot para una columna especificada y cambiarle el nombre a las nuevas.   
df_final_customer_phones = df_customer_basics_phones.groupBy("customer_id").pivot("row_num").agg(first('phone_number'))
df_final_customer_phones = df_final_customer_phones.withColumnRenamed("1", 'phone_number' + "_1") \
       .withColumnRenamed("2", 'phone_number' + "_2") \
       .withColumnRenamed("3", 'phone_number' + "_3") 

In [ ]:
df_final_customer_phones.show()

+-----------+-------------------+------------------+--------------+
|customer_id|     phone_number_1|    phone_number_2|phone_number_3|
+-----------+-------------------+------------------+--------------+
|   00000442| +54 297 15 5296284|+54 297 15 4729337|          null|
|   00001419| +54 11 15 66793207|              null|          null|
|   00001939| +54 11 15 38700150|              null|          null|
|   00002575|               null|              null|          null|
|   00002707|   +54 299  4477116|              null|          null|
|   00002790|   +54 342  4883620|+54 342 15 4662478|          null|
|   00004287|+54 3541 15 5988799|              null|          null|
|   00004724| +54 11 15 24084447|              null|          null|
|   00007932| +54 385 15 4065887|              null|          null|
|   00011850| +54 11 15 25781080|+54 3489 15 493578|          null|
|   00012051| +54 3401 15 534381|              null|          null|
|   00012611|   +54 11  43443165|+54 11 15 61141

## **FILTRADO Y REDUCCION DE DATOS --> ADDRESS (BBVAW2T1-28)**

In [ ]:
#Hago una ventana por el customer_id y ordenado por la ultima actualizacion 
w = Window.partitionBy(df_customer_basics_address['customer_id']).orderBy("last_change_date")

In [ ]:
#Agrego columna que cuente la cantidad de veces que se repite el ID
df_customer_basics_address = df_customer_basics_address.withColumn("row_num", row_number().over(w))

In [ ]:
#Corto el dataframe quedandome con las ultimas 3 actualizaciones
df_customer_basics_address = df_customer_basics_address.filter(col("row_num") <= 3)

In [ ]:
#Reacomodo la tabla con las columnas de interes.
df_customer_basics_address = df_customer_basics_address.select('customer_id', 'full_address', 'row_num')
df_customer_basics_address.show(10)

+-----------+--------------------+-------+
|customer_id|        full_address|row_num|
+-----------+--------------------+-------+
|   00000442|AGUERO           ...|      1|
|   00001419|ALTE G BROWN     ...|      1|
|   00001939|JOSE HERNANDEZ   ...|      1|
|   00002575|                null|      1|
|   00002707|GUATEMALA        ...|      1|
|   00002790|AV CORDOBA       ...|      1|
|   00004287|CNEL DE MARINA RO...|      1|
|   00004724|PJE MUTUALISMO   ...|      1|
|   00007932|AV FELIX DE OLAZA...|      1|
|   00011850|JUNCAL           ...|      1|
+-----------+--------------------+-------+
only showing top 10 rows



In [ ]:
#creo una funcion pivot para una columna especificada y cambiarle el nombre a las nuevas.
  
df_final_customer_address = df_customer_basics_address.groupBy("customer_id").pivot("row_num").agg(first('full_address'))
df_final_customer_address = df_final_customer_address.withColumnRenamed("1", 'full_address' + "_1") \
       .withColumnRenamed("2", 'full_address' + "_2") \
       .withColumnRenamed("3", 'full_address' + "_3")

In [ ]:
df_final_customer_address.show()

+-----------+--------------------+--------------+--------------+
|customer_id|      full_address_1|full_address_2|full_address_3|
+-----------+--------------------+--------------+--------------+
|   00000442|AGUERO           ...|          null|          null|
|   00001419|ALTE G BROWN     ...|          null|          null|
|   00001939|JOSE HERNANDEZ   ...|          null|          null|
|   00002575|                null|          null|          null|
|   00002707|GUATEMALA        ...|          null|          null|
|   00002790|AV CORDOBA       ...|          null|          null|
|   00004287|CNEL DE MARINA RO...|          null|          null|
|   00004724|PJE MUTUALISMO   ...|          null|          null|
|   00007932|AV FELIX DE OLAZA...|          null|          null|
|   00011850|JUNCAL           ...|          null|          null|
|   00012051|AV GRAL JOSE MARI...|          null|          null|
|   00012611|ALBERTO LARROQUE ...|          null|          null|
|   00012991|            

## **FILTRADO Y REDUCCION DE DATOS --> EMAILS (BBVAW2T1-29)**

In [ ]:
#Filtrado y reduccion de datos --> EMAILS
w = Window.partitionBy(df_customer_basics_email['customer_id']).orderBy("last_change_date")

In [ ]:
df_customer_basics_email = df_customer_basics_email.withColumn("row_num", row_number().over(w))

In [ ]:
df_customer_basics_email = df_customer_basics_email.filter(col("row_num") <= 3)

In [ ]:
df_customer_basics_email.show(10)

+-----------+--------------------+----------------+-------+
|customer_id|          email_desc|last_change_date|row_num|
+-----------+--------------------+----------------+-------+
|   00000442|                null|            null|      1|
|   00001419|EMILIA.RUBIANES@H...|      2015-10-08|      1|
|   00001419|  JU_LY1@HOTMAIL.COM|      2019-05-11|      2|
|   00001939|                null|            null|      1|
|   00002575|                null|            null|      1|
|   00002707|                null|            null|      1|
|   00002790|DIGITALIZACION@EE...|      1900-01-01|      1|
|   00002790| NOTIENE@HOIMAIL.COM|      2019-07-29|      2|
|   00004287|                null|            null|      1|
|   00004724|                null|            null|      1|
+-----------+--------------------+----------------+-------+
only showing top 10 rows



In [ ]:
#Hago primer pivot para dejar una fila por ID y tener cada email en columnas separadas
df_final_customer_email = df_customer_basics_email.groupBy("customer_id").pivot("row_num").agg(first("email_desc"))
df_final_customer_email = df_final_customer_email.withColumnRenamed("1", "email_1") \
       .withColumnRenamed("2", "email_2") \
       .withColumnRenamed("3", "email_3")
df_final_customer_email.show()

+-----------+--------------------+-------------------+-------+
|customer_id|             email_1|            email_2|email_3|
+-----------+--------------------+-------------------+-------+
|   00000442|                null|               null|   null|
|   00001419|EMILIA.RUBIANES@H...| JU_LY1@HOTMAIL.COM|   null|
|   00001939|                null|               null|   null|
|   00002575|                null|               null|   null|
|   00002707|                null|               null|   null|
|   00002790|DIGITALIZACION@EE...|NOTIENE@HOIMAIL.COM|   null|
|   00004287|                null|               null|   null|
|   00004724|                null|               null|   null|
|   00007932|                null|               null|   null|
|   00011850|                null|               null|   null|
|   00012051|                null|               null|   null|
|   00012611|                null|               null|   null|
|   00012991|                null|               null| 

## **AGREGAR COLUMNA CONTACT TYPE A LOS DATAFRAME (BBVAW2T1-30)**

In [ ]:
df_final_customer_address = df_final_customer_address.withColumn("Contact_type", lit('Address'))
df_final_customer_email = df_final_customer_email.withColumn("Contact_type", lit('Email'))
df_final_customer_phones = df_final_customer_phones.withColumn("Contact_type", lit('Phone'))

In [ ]:
#Reemplazo los valores en Tipo de Contacto que son null
df_final_customer_address = df_final_customer_address.withColumn("Contact_type", F.when(F.col("full_address_1").isNull(), F.lit(" ")).otherwise(F.lit("address")))
df_final_customer_phones = df_final_customer_phones.withColumn("Contact_type", F.when(F.col("phone_number_1").isNull(), F.lit(" ")).otherwise(F.lit("phone")))
df_final_customer_email = df_final_customer_email.withColumn("Contact_type", F.when(F.col("email_1").isNull(), F.lit(" ")).otherwise(F.lit("email")))

In [ ]:
df_final_customer_address.show()

+-----------+--------------------+--------------+--------------+------------+
|customer_id|      full_address_1|full_address_2|full_address_3|Contact_type|
+-----------+--------------------+--------------+--------------+------------+
|   00000442|AGUERO           ...|          null|          null|     address|
|   00001419|ALTE G BROWN     ...|          null|          null|     address|
|   00001939|JOSE HERNANDEZ   ...|          null|          null|     address|
|   00002575|                null|          null|          null|            |
|   00002707|GUATEMALA        ...|          null|          null|     address|
|   00002790|AV CORDOBA       ...|          null|          null|     address|
|   00004287|CNEL DE MARINA RO...|          null|          null|     address|
|   00004724|PJE MUTUALISMO   ...|          null|          null|     address|
|   00007932|AV FELIX DE OLAZA...|          null|          null|     address|
|   00011850|JUNCAL           ...|          null|          null|

## **AGREGAR COLUMNA DE JSON A TELEFONO (BBVAW2T1-31)**

In [ ]:
df_final_customer_phones = df_final_customer_phones.withColumn("phone_number_1_to_json", F.when(F.col("phone_number_1").isNull(), F.lit("sin_dato")).otherwise(F.col("phone_number_1")))
df_final_customer_phones = df_final_customer_phones.withColumn("phone_number_1_json", F.to_json(F.struct("phone_number_1_to_json")))

In [ ]:
df_final_customer_phones.count()

1614

## **AGREGAR COLUMNA DE JSON A DOMICILIO (BBVAW2T1-32)** 

In [ ]:
df_final_customer_address = df_final_customer_address.withColumn("full_address_1_to_json", F.when(F.col("full_address_1").isNull(), F.lit("sin_dato")).otherwise(F.col("full_address_1")))
df_final_customer_address = df_final_customer_address.withColumn("full_address_1_json", F.to_json(F.struct("full_address_1_to_json")))

In [ ]:
df_final_customer_email.count()

1614

In [ ]:
df_final_customer_address.select(F.collect_list('full_address_1_json')).first()[0]

['{"full_address_1_to_json":"AGUERO                   0000842 PB       B      CAPITAL FEDERAL                112233114455                   01171"}',
 '{"full_address_1_to_json":"ALTE G BROWN             0001074 1        A      MORON                          MORON                          01708"}',
 '{"full_address_1_to_json":"JOSE HERNANDEZ           0002045 1        C      CIUDAD AUTONOMA BUENOS AIRES   CAPITAL FEDERAL                01426"}',
 '{"full_address_1_to_json":"sin_dato"}',
 '{"full_address_1_to_json":"GUATEMALA                0001846                 VIRREYES                       112233114455                   01646"}',
 '{"full_address_1_to_json":"AV CORDOBA               0002206 4        30     CIUDAD AUTONOMA BUENOS AIRES   CAPITAL FEDERAL                01120"}',
 '{"full_address_1_to_json":"CNEL DE MARINA ROSALES   0002750 5        C      OLIVOS                         VICENTE LOPEZ                  01636"}',
 '{"full_address_1_to_json":"PJE MUTUALISMO           0002

## **JOIN TABLAS DE CONTACTOS (BBVAW2T1-33)**


In [ ]:
#cambiando el primer campo podemos hacer el join de las tres bases.
df_final_customer_phones = df_final_customer_phones.withColumnRenamed('customer_id','customer_id_B')

In [ ]:
df_final = df_final_customer_phones.join(df_final_customer_email, df_final_customer_phones.customer_id_B == df_final_customer_email.customer_id, how = "left")
df_final = df_final.drop("customer_id")
df_final = df_final.join(df_final_customer_address, df_final.customer_id_B == df_final_customer_address.customer_id, how = "left")
df_final = df_final.drop("customer_id")
df_final = df_final.withColumnRenamed('customer_id_B','customer_id')

In [ ]:
df_final.show()

+-----------+-------------------+------------------+--------------+------------+----------------------+--------------------+--------------------+-------------------+-------+------------+--------------------+--------------+--------------+------------+----------------------+--------------------+
|customer_id|     phone_number_1|    phone_number_2|phone_number_3|Contact_type|phone_number_1_to_json| phone_number_1_json|             email_1|            email_2|email_3|Contact_type|      full_address_1|full_address_2|full_address_3|Contact_type|full_address_1_to_json| full_address_1_json|
+-----------+-------------------+------------------+--------------+------------+----------------------+--------------------+--------------------+-------------------+-------+------------+--------------------+--------------+--------------+------------+----------------------+--------------------+
|   00000442| +54 297 15 5296284|+54 297 15 4729337|          null|       phone|    +54 297 15 5296284|{"phone_numb

## **RESGUARDAR 3 CONTACTOS DE CLIENTES (BBVAW2T1-34)**

In [ ]:
# Crear las nuevas columnas con valores no nulos de las columnas de contactos, priorizando Telefono, Mail o dirección
df_final = df_final.withColumn("contacto_1", coalesce("phone_number_1", "phone_number_2", "phone_number_3", "email_1", "email_2", "email_3", "full_address_1", "full_address_1", "full_address_1"))
df_final = df_final.withColumn("contacto_2", coalesce("phone_number_2", "phone_number_3", "email_1", "email_2", "email_3", "full_address_1", "full_address_2", "full_address_3"))
df_final = df_final.withColumn("contacto_3", coalesce("phone_number_3", "email_1", "email_1", "email_1", "full_address_1", "full_address_2", "full_address_3", 
                                        expr("coalesce(email_1, email_2, email_3, full_address_1, full_address_2, full_address_3)")))

df_final = df_final.withColumn("contacto_2", 
                               when(col("contacto_2").isin(col("contacto_1")), None)
                               .otherwise(col("contacto_2")))
df_final = df_final.withColumn("contacto_3", 
                               when(col("contacto_3").isin(col("contacto_1"), col("contacto_2")), None)
                               .otherwise(col("contacto_3")))

df_final.show()

+-----------+-------------------+------------------+--------------+------------+----------------------+--------------------+--------------------+-------------------+-------+------------+--------------------+--------------+--------------+------------+----------------------+--------------------+-------------------+--------------------+--------------------+
|customer_id|     phone_number_1|    phone_number_2|phone_number_3|Contact_type|phone_number_1_to_json| phone_number_1_json|             email_1|            email_2|email_3|Contact_type|      full_address_1|full_address_2|full_address_3|Contact_type|full_address_1_to_json| full_address_1_json|         contacto_1|          contacto_2|          contacto_3|
+-----------+-------------------+------------------+--------------+------------+----------------------+--------------------+--------------------+-------------------+-------+------------+--------------------+--------------+--------------+------------+----------------------+-------------

## **VISTA TEMPORAL DE CONTACTOS --> BBVAW2T1-35**

In [ ]:
df_final.createOrReplaceTempView("contacts_view")
spark.sql("SELECT * FROM contacts_view").show()

## **VISTA TEMPORAL DE CLIENTES --> BBVAW2T1-36**

In [ ]:
df_customer_basics.createOrReplaceTempView("customer_basics_view")
spark.sql("SELECT * FROM customer_basics_view").show(2000)

## **VISTA TEMPORAL DE ESTADO CIVIL --> BBVAW2T1-37**

In [ ]:
df_marital_status_type.createOrReplaceTempView("marital_status_type_view")
spark.sql("SELECT * FROM marital_status_type_view").show()

## **VISTA TEMPORAL DE NACIONALIDAD --> BBVAW2T1-38/62**





In [ ]:
df_nationality.createOrReplaceTempView("nationality_view")
spark.sql("SELECT * FROM nationality_view").show(1000)